<a href="https://colab.research.google.com/github/yhp2205/SQL/blob/main/ch_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 10강 다면적인 축을 사용해 데이터 집약하기

In [1]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


EC 사이트를 가정하여 샘플 데이터를 입력하도록 하겠습니다. 

In [2]:
%%bigquery --project mygcpproject-340112
DROP TABLE IF EXISTS sqldata.purchase_detail_log;
CREATE TABLE sqldata.purchase_detail_log(
    dt           string
  , order_id     integer
  , user_id      string
  , item_id      string
  , price        integer
  , category     string
  , sub_category string
);

INSERT INTO sqldata.purchase_detail_log
VALUES
    ('2017-01-18', 48291, 'usr33395', 'lad533', 37300,  'ladys_fashion', 'bag')
  , ('2017-01-18', 48291, 'usr33395', 'lad329', 97300,  'ladys_fashion', 'jacket')
  , ('2017-01-18', 48291, 'usr33395', 'lad102', 114600, 'ladys_fashion', 'jacket')
  , ('2017-01-18', 48291, 'usr33395', 'lad886', 33300,  'ladys_fashion', 'bag')
  , ('2017-01-18', 48292, 'usr52832', 'dvd871', 32800,  'dvd'          , 'documentary')
  , ('2017-01-18', 48292, 'usr52832', 'gam167', 26000,  'game'         , 'accessories')
  , ('2017-01-18', 48292, 'usr52832', 'lad289', 57300,  'ladys_fashion', 'bag')
  , ('2017-01-18', 48293, 'usr28891', 'out977', 28600,  'outdoor'      , 'camp')
  , ('2017-01-18', 48293, 'usr28891', 'boo256', 22500,  'book'         , 'business')
  , ('2017-01-18', 48293, 'usr28891', 'lad125', 61500,  'ladys_fashion', 'jacket')
  , ('2017-01-18', 48294, 'usr33604', 'mem233', 116300, 'mens_fashion' , 'jacket')
  , ('2017-01-18', 48294, 'usr33604', 'cd477' , 25800,  'cd'           , 'classic')
  , ('2017-01-18', 48294, 'usr33604', 'boo468', 31000,  'book'         , 'business')
  , ('2017-01-18', 48294, 'usr33604', 'foo402', 48700,  'food'         , 'meats')
  , ('2017-01-18', 48295, 'usr38013', 'foo134', 32000,  'food'         , 'fish')
  , ('2017-01-18', 48295, 'usr38013', 'lad147', 96100,  'ladys_fashion', 'jacket')
 ;


""


위 데이터를 이용하여 카테고리의 소계와 총계를 한 번에 출력해보도록 하겠습니다. 계층별로 집계한 결과를 같은 컬럼이 되게끔 변환한 후 하나의 테이블로 합치면 됩니다. 

In [3]:
%%bigquery --project mygcpproject-340112
  -- 10.1 카테고리별 매출과 소계를 동시에 구하는 쿼리
WITH
sub_category_amount AS (
  -- 소 카테고리의 매출 집계하기
  SELECT
      category    AS category
    , sub_category AS sub_category
    , SUM(price) AS amount
  FROM
    sqldata.purchase_detail_log
  GROUP BY
    category, sub_category
)
, category_amount AS (
    -- 대 카테고리의 매출 집계하기
  SELECT
      category
    , 'all' AS sub_category
    , SUM(price) AS amount
  FROM
    sqldata.purchase_detail_log
  GROUP BY
    category
)
, total_amount AS (
    -- 전체 매출 집계하기
  SELECT
      'all' AS category
    , 'all' AS sub_category
    , SUM(price) AS amount
  FROM
    sqldata.purchase_detail_log
)
          SELECT category, sub_category, amount FROM sub_category_amount
UNION ALL SELECT category, sub_category, amount FROM category_amount
UNION ALL SELECT category, sub_category, amount FROM total_amount
;

,category,sub_category,amount
0,ladys_fashion,all,497400
1,dvd,all,32800
2,game,all,26000
3,outdoor,all,28600
4,book,all,53500
5,cd,all,25800
6,mens_fashion,all,116300
7,food,all,80700
8,ladys_fashion,bag,127900
9,ladys_fashion,jacket,369500


하나의 쿼리를 사용하여 카테고리별 소계와 전체의 총계를 동시에 계산해보았습니다. PostgreSQL이나 HIVE와 같은 곳에서는 ROLLUP 을 사용하여 UNION ALL을 사용하지 않고도 테이블을 결합할 수 있습니다. 다만 Bigquery에서는 지원하지 않으니 넘어가도록 하겠습니다.

다음으로 ABC 분석을 해보도록 하겠습니다. 중요도에 따라 상품을 A, B, C로 나누어 분석하는 방법입니다.  
다음 코드에서는 2017년 1월의 구매로그를 기반으로 매출 구성비누계와 ABC 등급을 계산하는 쿼리입니다.  
1. 원하는 시간대의 구매 로그를 압축하고  
2. 상품 카테고리마다의 매출을 계산한 후  
3. 전체 매출에 대해 항목별 매출 구성비와 구성비 누계를 계산합니다  
4. 구성비 누계를 기준으로 0~70%, 70~90%, 90~100% 이렇게 등급을 나누도록 하겠습니다. 

In [4]:
%%bigquery --project mygcpproject-340112
  -- 10.3 매출 구성비 누계와 ABC 등급을 계산하는 쿼리
WITH
monthly_sales AS (
  SELECT
    category
    -- 항목별 매출 계산하기
    , SUM(price) AS amount
  FROM
    sqldata.purchase_detail_log
  -- 대상 1개월 동안의 로그를 조건으로 걸기
  WHERE
    dt BETWEEN '2017-01-01' AND '2017-01-31'
  GROUP BY
    category
)
, sales_composition_ratio AS (
  SELECT
      category
    , amount

    -- 구성비 : 100.0 * <항목별 매출> / <전체 매출>
    , 100.0 * amount / SUM(amount) OVER() AS composition_ratio

    -- 구성비누계 : 100.0 * <항목별 구계 매출> / <전체 매출>
    , 100.0 * SUM(amount) OVER(ORDER BY amount DESC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)
     / SUM(amount) OVER() AS cumulative_ratio
  FROM
    monthly_sales
)
SELECT
    *
  -- 구성비 누계 범위에 따라 순위 붙이기
  , CASE
    WHEN cumulative_ratio BETWEEN  0 AND 70 THEN 'A'
    WHEN cumulative_ratio BETWEEN 70 AND 90 THEN 'B'
    WHEN cumulative_ratio BETWEEN 90 AND 100 THEN 'C'
  END AS abc_rank
FROM
  sales_composition_ratio
ORDER BY
  amount DESC
;

,category,amount,composition_ratio,cumulative_ratio,abc_rank
0,ladys_fashion,497400,57.763326,57.763326,A
1,mens_fashion,116300,13.505981,71.269307,B
2,food,80700,9.371734,80.641041,B
3,book,53500,6.212983,86.854024,B
4,dvd,32800,3.809081,90.663105,C
5,outdoor,28600,3.321333,93.984439,C
6,game,26000,3.019394,97.003832,C
7,cd,25800,2.996168,100.000000,C
